# Introduction to variantannotation: package for the aggregation of genomic variant data 

#### Author: C. Mazzaferro & Kathleen Fisch
#### Email: cmazzafe@ucsd.edu
#### Date: January 2017
 
## Outline of Notebook
<a id = "toc"></a>
1. <a href = "#background">Background</a>
2. <a href = "#setup">Set Up File and Libraries</a>
3. <a href = "#ANNOVAR">Run Annovar</a>
4. <a href = "#export1">Export Data To MongoDB: Method 1</a>
5. <a href = "#export2">Export Data To MongoDB: Method 2</a>
6. <a href = "#filter">Variant Filtering</a>
    * <a href = "#tumorvars">Rare Tumor Variant Filter</a>
    * <a href = "#diseasevars">Rare Diesease Variant Filter</a>
    * <a href = "#caddvars">CADD PHRED High Impact Variants</a>
    * <a href = "#own">Create Your Own Filter</a>
7. <a href = "#export">Export CSV and VCF files</a>

<a id = "background"></a>
## Background

This notebook will walk you through the steps of how variants coming from a VCF can be annotated efficiently and thoroughly using the package VAPr. In particular, the package is aimed at providing a way of retrieving variant information using [ANNOVAR](http://annovar.openbioinformatics.org/en/latest/) and [myvariant.info](myvariant.info) and consolidating it in conveninent formats. It is well-suited for bioinformaticians interested in aggregating variant information into a single database for ease of use and to provide higher analysis capabities. 

The aggregation is performed specifically by structuring the data in lists of python dictionaries, with each variant being described by a multi-level dictionary. The choice was made due to the inconsistencies that exist between data availability, and the necessity to store their information in a flexible manner. Further, this specific format permits its parsing to a MongoDb instance (dictionaries are the python representation of JSON objects), which enables the user to efficiently store, query and filter such data. 

Finally, the package also has the added functionality to create csv and vcf files from MongoDB. The class Filters allows the user to rapidly query data that meets certain criteria as a list of documents, and the class FileWriter can transform such list into more widely accepted formats such as vcf and csv files. It should be noted that here, the main differential the package offers is the ability to write these files preserving all the annotation data. In the vcf files, for instance, outputs will have a 'Otherinfo' column where all the data coming from ANNOVAR and myvariant.info is condensed (while still preserving its structure). For vcf files, outputs will have around ~120-200 columns, depending on the amount of variant data that can be retrieved from myvvariant.info. 

Having the data stored in a database offers a variety of benefits. In particular, it enables the user to set customized queries and rapidly iterate over a specific procedure and get maximum reproducibility. It also enables the storage of data coming from different sources, and its rapid access. 

**Notes on required software**

the following libraries will be installed upon installing VAPr:
- myvariant
- pymongo
- pyvcf

Other libraries that are needed, but should natively e installed on most OS: 

- Pandas
- Numpy

Further, a MongoDB database must be set up. Refer to the documentation page for more information. 
Similarly, ANNOVAR must be downloaded, alongside with its supporting databases (also listed on the documentation page).

# Flowchart Diagram

In [ ]:
import os
from IPython.display import Image, display, HTML
Image(filename=os.path.dirname(os.path.realpath('__file__')) + '/simpler.jpg') 

<a id = "setup"></a>
## Import libraries

`VAPr.base` is your interface with the annotation and storing tools offered in the package.

In [ ]:
from VAPr import base

<a id = "ANNOVAR"></a>
## Create an Annotation Project

The quickest way to get everything up and running is by creating a project where you specify the required paths. To start off, you will need to have your `vcf` files in a specific directory, and create a new, empty directory to which your annotated files will be written to. 

It is also required that you specify the location to which you dowloaded annovar. The folder where annovar lives looks like this:

    ... /annovar/
             annotate_variation.pl
             coding_change.pl
             convert2annovar.pl
             example/    
             humandb/
             retrieve_seq_from_fasta.pl
             table_annovar.pl
             variants_reduction.pl    

It currently supports the three different genome builds, `hg19`, `hg18`, `hg38`

**NOTE**: we can't provide the ANNOVAR package as it requires registration to be downloaded. It is, however, freely available [here](http://annovar.openbioinformatics.org/en/latest/user-guide/download/). Download, unzip and place it in whatever directory you'd like. Make sure you have enough space on disk (~15 GB for the datasets used for annotation).
### In case you'd want to run the annotations without having to worry about annovar, you can skip to <a href = "#export2">Export Data To MongoDB: Method 2</a>. Note that it will result in an easier process, but less data will be retrieved

In [ ]:
# Directory of input files to be annotated
IN_PATH = "/Volumes/Carlo_HD1/CCBB/VAPr_files/vcf_benchmark/"

# Output file directory
OUT_PATH = "/Volumes/Carlo_HD1/CCBB/VAPr_files/csv_benchmark/"

# Location of your annovar dowload. The folder should contain the following files/directories:
ANNOVAR_PATH = '/Volumes/Carlo_HD1/CCBB/annovar/'  

# Databse and Collection names (optional)
proj_data = {'db_name': 'VariantDBBenchmarkWES',
            'project_name': 'collect'}


Project = base.AnnotationProject(IN_PATH, 
                                 OUT_PATH, 
                                 ANNOVAR_PATH, 
                                 proj_data,  
                                 build_ver='hg19')

### Download Databses

Download annotation databases from Annovar and other repos. Files will be stored at .../annovar/humandb.  You may open that directory to check the progress: make sure all files are unzipped (VAPr does that automatically) before proceeding. Usually the database hg19_1000g2015aug.zip takes some time to be fully downloaded and unzipped (around 10 minutes).

In [ ]:
Project.download_dbs()

#### Check dowload status
This command will open the download directory so that you can check if all files are there and unzipped. Make sure all files are in .txt and .idx format before proceeding

In [ ]:
import subprocess, os
subprocess.Popen(['open', os.path.join(ANNOVAR_PATH, 'humandb')], stdout=subprocess.PIPE)

## Run Annovar 

The next command will run the annotations on each file and store them in a directory specified by the file's sample name. Don't worry, VAPr figures out automatically the sampele name, and if there are more than one sample in a single file. The csv files can then be processed and integrated with the data coming from myvariant.info. 
This command may take a some time to run (5-30 minutes for each file depending on file size). Feel free to open the OUT_PATH directory to see the annotation files being created on the fly.

In [ ]:
Project.run_annovar()

<a id = "export1"></a>
## Export Annovar, MyVariant data to MongoDB

With this method, data from annovar (as a txt file) will be obtained 1000 lines at a time, one file at a time.

As soon as you run the scripts from VAPr, variant data will be retrieved from myvariant.info and the data will automatically be integrated and stored to MongoDB. Database and collection name should be specified, and there must be a running MongoDB connection. The script will set up a client to communicate between python (through pymongo) and the the database.

In general, the shell command:

`mongod --dbpath ../data/db`  

Where data/db is the designated location where the data will be stored, will initiate MongoDB. After this, the script should store data to the directory automatically.
For pymongo, and more information on how to set up a Mongo Database: https://docs.mongodb.com/getting-started/python/

In [ ]:
%prun Project.annotate_and_save()

## Code profiling
`%prun` serves to  profile the code's performance. The output tells you the runtime, and number of function calls. Feel free to remove it from the cell.

### 210215396 function calls  in 905.227 seconds  

## Speed Up Annotation With Parallel Processing

Alternatively, you may want to store a number of files to MongoDB, which may take a long time. This can be sped up by running multiple annotations at the same time. The syntax is pretty much the same. You just need to specify the number of cores to be used. 

In [ ]:
%prun Project.parallel_annotation_and_saving(n_processes=6)

## Code profiling

### 95889 function calls in 238.308 seconds. 4x speedup. 

<a id = "export2"></a>
## Export MyVariant data to MongoDB

This will export variant data from a vcf file to mongo, annotating it with MyVariant.info solely. It does not require annovar, and may result in some empty variants. 

- Easier to run, doesn't require annovar.
- Will however be incomplete (some variants will have no information).

In [1]:
from VAPr import base

# Directory of input files to be annotated
IN_PATH = "/Volumes/Carlo_HD1/CCBB/VAPr_files/vcf_benchmark/"

# Output file directory
OUT_PATH = "/Volumes/Carlo_HD1/CCBB/VAPr_files/csv_benchmark/"

# Location of your annovar dowload. The folder should contain the following files/directories:
ANNOVAR_PATH = '/Volumes/Carlo_HD1/CCBB/annovar/'  

# Databse and Collection names (optional)
proj_data = {'db_name': 'VariantDBBenchmarkWES',
            'project_name': 'myvariant_only_collection'}


Project = base.AnnotationProject(IN_PATH, 
                                 OUT_PATH, 
                                 ANNOVAR_PATH, 
                                 proj_data,  
                                 build_ver='hg19')

/Users/carlomazzaferro/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1041: UserWarning: Duplicate key in file "/Users/carlomazzaferro/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


INFO:root:Csv output dir /Volumes/Carlo_HD1/CCBB/VAPr_files/csv_benchmark/sample_BC001 for sample exists, moving files there
INFO:root:Vcf input dir /Volumes/Carlo_HD1/CCBB/VAPr_files/vcf_benchmark/sample_BC001 for sample exists, moving files there
(True, '/Volumes/Carlo_HD1/CCBB/VAPr_files/vcf_benchmark/sample_BC001/BC001.final.vcf')


In [ ]:
Project.quick_annotate_and_save(n_processes=8)

 50%|█████     | 26/52 [04:15<04:33, 10.53s/it]

## Create output files by filtering
Create 4 output files: annotated vcf, annotated csv, filtered vcf, filtered csv. 
All files will contain information from myvariant and ANNOVAR; the filtered ones will be much smaller in size.
### See "Notes on Variant Filtering & Output Files" for more info 

In [ ]:
#Apply filter(s).

from VAPr import MongoDB_querying, file_writer
importlib.reload(MongoDB_querying)


#Define names/paths
filepath = '/Volumes/Carlo_HD1/'

#The input gzipped vcf file is required to create a filtered one.
in_vcf_file = filepath + "/normal_targeted_seq.vcf.gz"

out_unfiltered_vcf_file = filepath + "/_unfilterd_vcf_annotated.vcf"
out_unfiltered_csv_file = filepath + "/_unfiltered_csv_annotated.csv"

rare_cancer_variants_csv = filepath + "/_rare_cancer_vars.csv"
rare_cancer_variants_vcf = filepath + "/_rare_cancer_vars.vcf"


### Query Rare Cancer Variants

In [ ]:
from VAPr import MongoDB_querying, file_writer
importlib.reload(MongoDB_querying)

In [ ]:
filter_collection = MongoDB_querying.Filters(db_name, collection_name)  #Filter Object

#Three different filters. Let's use the first one.
rare_cancer_variants = filter_collection.rare_cancer_variant()

In [ ]:
#Create writer object that pulls data from MongoDB
my_writer = file_writer.FileWriter(db_name, collection_name)

#Write collection to csv and vcf
my_writer.generate_unfiltered_annotated_csv(out_unfiltered_csv_file)
my_writer.generate_unfiltered_annotated_vcf(in_vcf_file, out_unfiltered_vcf_file)

#cancer variants filtered files
my_writer.generate_annotated_csv(rare_cancer_variants, rare_cancer_variants_csv)
my_writer.generate_annotated_vcf(rare_cancer_variants, in_vcf_file, rare_cancer_variants_vcf)

### Other methods and capabilities

In [ ]:
#Obtain list of variant ID's from a vcf file
open_file = myvariant_parsing_utils.VariantParsing()
variant_list = open_file.get_variants_from_vcf(vcf_file)

#Or, if file is too large to be held in memory, iterate over it. 
variant_list = open_file.get_variants_from_vcf_chunk(vcf_file, 10000, 0)

#Since data is a 

<a id = "filter"></a>
## Notes on Variant Filtering & Output Files

Here we implement three different filters that allow for the retrieval of specific variants. The filters are implemented as MongoDB queries, and are designed to provie the user with a set of relevant variants. In case the user would like to define its own querying, a template is provided. 
The output of the queries is a list of dictionaries (JSON documents), where each dictionary contains data reltive to one variant. 

Further, the package allows the user to parse these variants into an annotated csv or vcf file. 
If needed, annotated, unfiltered vcf and csv files can also be created. They will have the same length (number of variants) as the original files, but will contain much more complete annotation data coming from myvariant.info and ANNOVAR databases. 

To create a csv file, just the filtered output is needed. To create an annotated vcf file, a tab indexed file (.tbi) file is needed (see comments in  section Create unfiltered annotated vcf and csv files at the end of this page). This can be created using tabix.  

First, the file needs to be compressed:

From the command line, running:

`bgzip -c input_file.vcf > input_file.vcf.gz`

returns `input_vcf_file.vcf.gz`

and running 

`tabix input_vcf_file.vcf.gz`

will return: `input_vcf_file.vcf.gz.tbi`



<a id = "tumorvars"></a>
## Filter #1: specifying cancer-specifc rare variants

 - filter 1: ThousandGenomeAll < 0.05 or info not available
 - filter 2: ESP6500siv2_all < 0.05 or info not available
 - filter 3: cosmic70 information is present
 - filter 4: Func_knownGene is exonic, splicing, or both
 - filter 5: ExonicFunc_knownGene is not "synonymous SNV"
 - filter 6: Read Depth (DP) > 10

In [ ]:
filepath = '/data/ccbb_internal/interns/Carlo'

In [ ]:
#Create output files (if needed): specify name of files and path 
rare_cancer_variants_csv = filepath + "/tumor_rna_rare_cancer_vars_csv.csv"
rare_cancer_variants_vcf = filepath + "/tumor_rna_rare_cancer_vars_vcf.vcf"
input_vcf_compressed = filepath + '/test_vcf/Tumor_RNAseq_variants.vcf.gz'

#Apply filter.
filter_collection = MongoDB_querying.Filters(db_name, collection_name)
rare_cancer_variants = filter_collection.rare_cancer_variant()


<a id = "rarevars"></a>
## Filter #2: specifying rare disease-specifc (rare) variants

- filter 1: ThousandGenomeAll < 0.05 or info not available
- filter 2: ESP6500siv2_all < 0.05 or info not available
- filter 3: cosmic70 information is present
- filter 4: Func_knownGene is exonic, splicing, or both
- filter 5: ExonicFunc_knownGene is not "synonymous SNV"
- filter 6: Read Depth (DP) > 10
- filter 7: Clinvar data is present 

In [ ]:
#Apply filter.
filter_collection = MongoDB_querying.Filters(db_name, collection_name)
rare_disease_variants = filter_collection.rare_disease_variant()

Zero variants found. Writing a csv output won't make much sense. You can still customize the filters the way you'd like, as you can see on 'Create your own filter'.

<a id = "caddvars"></a>
## Filter #3: specifying rare disease-specifc (rare) variants with high impact
- filter 1: ThousandGenomeAll < 0.05 or info not available
- filter 2: ESP6500siv2_all < 0.05 or info not available
- filter 3: cosmic70 information is present
- filter 4: Func_knownGene is exonic, splicing, or both
- filter 5: ExonicFunc_knownGene is not "synonymous SNV"
- filter 6: Read Depth (DP) > 10
- filter 7: Clinvar data is present 
- filter 8: cadd.phred > 10

In [ ]:
#Apply filter.
filter_collection = MongoDB_querying.Filters(db_name, collection_name)
rare_high_impact_variants = filter_collection.rare_high_impact_variants()

<a id = "own"></a>
## Create your own filter

As long as you have a MongoDB instance running, filtering can be perfomed trough pymongo as shown by the code below. If a list is intended to be created from it, simply add: `filter2 = list(filter2)`

If you'd like to customize your filters, a good idea would be to look at the available fields to be filtered. Looking at the myvariant.info [documentation](http://docs.myvariant.info/en/latest/doc/data.html), you can see what are all the fields avaialble and can be used for filtering. 

In [ ]:
from pymongo import MongoClient

client = MongoClient()
db = client.My_Variant_Database
collection = db.ANNOVAR_MyVariant_chunks

filtered = collection.find({"$and": [
                                   {"$or": [{"esp6500siv2_all": {"$lt": 0.1}}, {"esp6500siv2_all": {"$exists": False}}]},
                                   {"$or": [{"func_knowngene": "exonic"}, {"func_knowngene": "splicing"}]},
                                   {"genotype.filter_passing_reads_count": {"$gte": 1}},
                                   {"cosmic70": {"$exists": True}},
                                   {"1000g2015aug_all": {"$exists": True}}
                         ]})
